In [1]:
from utils import *
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.decomposition import PCA
from collections import Counter

In [2]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")

In [3]:
selected_feature_model = "fc_fd"

m = int(input("Enter value of m: "))
if m < 1:
    raise ValueError("m should be a positive integer")

selected_image_id = int(
    input("Enter odd image ID: ")
)

In [4]:
dataset = loadDataset(Caltech101)

all_images = list(fd_collection.find())
odd_image_ids = [img["image_id"] for img in all_images if img["image_id"] % 2 == 0]

even_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 == 0]
odd_image_labels = [img["true_label"] for img in all_images if img["image_id"] % 2 != 0]

even_feature_vectors = [np.array(img[selected_feature_model]).flatten() for img in all_images if img["image_id"] % 2 == 0]
odd_feature_vectors = [np.array(img[selected_feature_model]).flatten() for img in all_images if img["image_id"] % 2 != 0]
feature_vector_labels = [img['true_label'] for img in all_images]

pca = PCA(n_components=150)
even_feature_vectors = pca.fit_transform(even_feature_vectors)
odd_feature_vectors = pca.fit_transform(odd_feature_vectors)

odd_len = odd_feature_vectors.shape[0]
even_len = even_feature_vectors.shape[0]

# for i in range(n):
#     selected_image_id = i * 2 + 1
#     selected_fd = np.expand_dims(np.array(odd_image_fds[i]), axis = 0)
#     print(f"Reference image: {selected_image_id}, similar images: {neigh.predict(selected_fd)}")

In [5]:
# class KDNode:
#     def __init__(self, point, left=None, right=None):
#         self.point = point
#         self.left = left
#         self.right = right

# def build_kd_tree(data, depth=0):
#     if not data.all():
#         return None
#     k = len(data[0])
#     axis = depth % k
#     data.sort(key=lambda x: x[axis])
#     median = len(data) // 2
#     return KDNode(
#         point=data[median],
#         left=build_kd_tree(data[:median], depth + 1),
#         right=build_kd_tree(data[median + 1:], depth + 1)
#     )

# def euclidean_distance(point1, point2):
#     return np.linalg.norm(np.array(point1) - np.array(point2))

# def k_nn_search(tree, query, k, depth=0):
#     if not tree:
#         return []
#     k = len(query)
#     axis = depth % k
#     next_branch = None
#     opposite_branch = None
#     if query[axis] < tree.point[axis]:
#         next_branch = tree.left
#         opposite_branch = tree.right
#     else:
#         next_branch = tree.right
#         opposite_branch = tree.left
#     best = k_nn_search(next_branch, query, k, depth + 1)
#     best.append(tree.point)
#     best.sort(key=lambda x: euclidean_distance(x, query))
#     if len(best) > k:
#         best = best[:k]
#     if abs(query[axis] - tree.point[axis]) < euclidean_distance(best[-1], query):
#         best += k_nn_search(opposite_branch, query, k, depth + 1)
#     return best

In [8]:
# from collections import Counter
# from statistics import mode
# for i in range(odd_len):

#     distances = []

#     for j in range(even_len):
#         distances.append({"label": even_image_labels[j], "distance": euclidean_distance_measure(odd_feature_vectors[i], even_feature_vectors[j])})
    
#     sorted_distances = sorted(distances, key=lambda x: x['distance'])[:m]

#     labels = [sd["label"] for sd in sorted_distances]
#     print(labels)
    
#     print(f"Image ID: {i * 2 + 1} is similar to {max(set(labels), key = labels.count)}")

[4, 1, 2, 16, 41, 4, 41, 2, 4, 4, 2, 2, 41, 2, 2]
Image ID: 1 is similar to 2
[1, 1, 0, 1, 1, 0, 1, 1, 29, 0, 0, 29, 1, 1, 0]
Image ID: 3 is similar to 1
[0, 0, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1, 1, 0, 2]
Image ID: 5 is similar to 0
[0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1]
Image ID: 7 is similar to 1
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1]
Image ID: 9 is similar to 1
[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
Image ID: 11 is similar to 1
[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0]
Image ID: 13 is similar to 1
[18, 1, 1, 59, 1, 59, 1, 87, 90, 1, 1, 18, 18, 91, 1]
Image ID: 15 is similar to 1
[1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Image ID: 17 is similar to 1
[0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 0, 1]
Image ID: 19 is similar to 0
[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
Image ID: 21 is similar to 1
[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1]
Image ID: 23 is similar to 1
[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]
Image ID: 25 is similar to 0
[0, 0, 1, 1, 1, 

In [18]:
import numpy as np

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
    
    def entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities))
    
    def information_gain(self, X, y, feature, threshold):
        left_idxs = X[:, feature] <= threshold
        right_idxs = ~left_idxs
        
        left_y = y[left_idxs]
        right_y = y[right_idxs]
        
        p_left = len(left_y) / len(y)
        p_right = len(right_y) / len(y)
        
        gain = self.entropy(y) - (p_left * self.entropy(left_y) + p_right * self.entropy(right_y))
        return gain
    
    def find_best_split(self, X, y):
        best_gain = 0
        best_feature = None
        best_threshold = None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self.information_gain(X, y, feature, threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        print("find_best_split")
        return best_feature, best_threshold
    
    def build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1 or depth == self.max_depth:
            return Node(value=np.argmax(np.bincount(y)))
        
        best_feature, best_threshold = self.find_best_split(X, y)
        
        if best_feature is None:
            return Node(value=np.argmax(np.bincount(y)))
        
        left_idxs = X[:, best_feature] <= best_threshold
        right_idxs = ~left_idxs
        
        left_subtree = self.build_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_subtree = self.build_tree(X[right_idxs], y[right_idxs], depth + 1)
        
        print("build_tree")
        return Node(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)
    
    def fit(self, X, y):
        X = np.array(X)  # Convert to NumPy array
        y = np.array(y)  # Convert to NumPy array
        self.tree = self.build_tree(X, y)
    
    def predict_instance(self, x, node):
        if node.value is not None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.predict_instance(x, node.left)
        else:
            return self.predict_instance(x, node.right)
    
    def predict(self, X):
        X = np.array(X)  # Convert to NumPy array
        predictions = []
        for x in X:
            pred = self.predict_instance(x, self.tree)
            predictions.append(pred)
        return np.array(predictions)

# Example usage:
# Assuming X_train and y_train are your training data, and X_test is your test data
# Create and fit the decision tree
tree = DecisionTree(max_depth=10)
tree.fit(even_feature_vectors, even_image_labels)

print("Trees formed")

# Make predictions on test data
predictions = tree.predict(odd_feature_vectors)
print(predictions)

find_best_split
find_best_split
find_best_split
find_best_split
find_best_split
build_tree
find_best_split
find_best_split
build_tree
find_best_split
find_best_split
find_best_split
build_tree
build_tree
build_tree
build_tree
build_tree
find_best_split
find_best_split
find_best_split
find_best_split
find_best_split
build_tree
find_best_split
build_tree
build_tree
build_tree
build_tree
find_best_split
find_best_split
find_best_split
find_best_split
build_tree
build_tree
find_best_split
build_tree
build_tree
find_best_split
find_best_split
find_best_split
find_best_split
build_tree
find_best_split
find_best_split
build_tree
build_tree
build_tree
find_best_split
find_best_split
find_best_split
build_tree
build_tree
find_best_split
find_best_split
build_tree
find_best_split
build_tree
build_tree
build_tree
build_tree
find_best_split
find_best_split
find_best_split
build_tree
find_best_split
find_best_split
build_tree
build_tree
build_tree
build_tree
build_tree
build_tree
build_tree
build_t